In [4]:
import requests
import pandas as pd

def create_hgvs(chrom, pos, ref, alt):
    return f"{chrom}:g.{pos}{ref}>{alt}"

def search_variation_id(hgvs_str):
    url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    params = {
        "db": "clinvar",
        "term": hgvs_str,
        "retmode": "json"
    }
    try:
        response = requests.get(url, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()
        idlist = data.get("esearchresult", {}).get("idlist", [])
        return idlist[0] if idlist else None
    except Exception as e:
        return None

def parse_vcf(file_path):
    records = []
    with open(file_path, "r") as vcf:
        for line in vcf:
            if line.startswith("#"):
                continue  # header satırlarını atla
            parts = line.strip().split("\t")
            chrom, pos, _, ref, alt = parts[0], parts[1], parts[2], parts[3], parts[4]
            records.append((chrom, pos, ref, alt))
    return records

# 📂 VCF dosyasını oku
vcf_path = "dna1_test.vcf"  # buraya kendi dosya adını yaz
variants = parse_vcf(vcf_path)

# 🔄 Her varyant için ClinVar ID bul
results = []
for chrom, pos, ref, alt in variants:
    hgvs = create_hgvs(chrom, pos, ref, alt)
    var_id = search_variation_id(hgvs)
    results.append({
        "CHROM": chrom,
        "POS": pos,
        "REF": ref,
        "ALT": alt,
        "HGVS": hgvs,
        "Variation ID": var_id or "Bulunamadı"
    })

# 📊 Excel'e yaz
df = pd.DataFrame(results)
df.to_excel("clinvar_variation_ids.xlsx", index=False)
print("✅ Excel dosyası oluşturuldu: clinvar_variation_ids.xlsx")


KeyboardInterrupt: 